In [1]:
pip install beautifulsoup4


The following command must be run outside of the IPython shell:

    $ pip install beautifulsoup4

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [2]:
pip install lxml


The following command must be run outside of the IPython shell:

    $ pip install lxml

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
pip install requests


The following command must be run outside of the IPython shell:

    $ pip install requests

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [6]:
tableHTML = soup.find('table', class_ = 'wikitable sortable')

In [7]:
table = []
for row in tableHTML.find_all('tr'):
    
    for entry in row.find_all('td'):
        table.append(entry.text)

In [8]:
def devideList(List, size):
    for i in range(0, len(List), size):  
        yield List[i:i + size]

In [9]:
table = list(devideList(table, 3))

In [10]:
import pandas as pd

In [11]:
neighbourhood = pd.DataFrame(table, columns = ["Postcode", "Borough", "Neighbourhood"])
neighbourhood.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [12]:
neighbourhood['Neighbourhood'].replace(regex=True,inplace=True,to_replace='\n',value=r'')
neighbourhood.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [13]:
neighbourhood.drop(neighbourhood.loc[neighbourhood['Borough']=='Not assigned'].index, inplace=True)
neighbourhood.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [14]:
x = neighbourhood.groupby('Postcode')['Neighbourhood'].apply(list)
x = x.reset_index(name='Neighbourhood')

In [15]:
x.head()

,Postcode,Neighbourhood
0,M1B,"[Rouge, Malvern]"
1,M1C,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,"[Guildwood, Morningside, West Hill]"
3,M1G,[Woburn]
4,M1H,[Cedarbrae]


In [16]:
x['Neighbourhood1'] = x.Neighbourhood.apply(', '.join)
x.head()

,Postcode,Neighbourhood,Neighbourhood1
0,M1B,"[Rouge, Malvern]","Rouge, Malvern"
1,M1C,"[Highland Creek, Rouge Hill, Port Union]","Highland Creek, Rouge Hill, Port Union"
2,M1E,"[Guildwood, Morningside, West Hill]","Guildwood, Morningside, West Hill"
3,M1G,[Woburn],Woburn
4,M1H,[Cedarbrae],Cedarbrae


In [17]:
x = pd.merge(x,neighbourhood, on='Postcode')
x.head()

,Postcode,Neighbourhood_x,Neighbourhood1,Borough,Neighbourhood_y
0,M1B,"[Rouge, Malvern]","Rouge, Malvern",Scarborough,Rouge
1,M1B,"[Rouge, Malvern]","Rouge, Malvern",Scarborough,Malvern
2,M1C,"[Highland Creek, Rouge Hill, Port Union]","Highland Creek, Rouge Hill, Port Union",Scarborough,Highland Creek
3,M1C,"[Highland Creek, Rouge Hill, Port Union]","Highland Creek, Rouge Hill, Port Union",Scarborough,Rouge Hill
4,M1C,"[Highland Creek, Rouge Hill, Port Union]","Highland Creek, Rouge Hill, Port Union",Scarborough,Port Union


In [18]:
x.drop(columns=['Neighbourhood_x','Neighbourhood_y'], inplace = True)
x = x.rename(columns={'Neighbourhood1':'Neighbourhood'})
x.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1B,"Rouge, Malvern",Scarborough
2,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
4,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough


In [19]:
x = x.drop_duplicates()
x.reset_index(drop=True, inplace = True)
x.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


In [20]:
FinalDf = x[['Postcode','Borough','Neighbourhood']]
FinalDf.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
FinalDf.shape

(103, 3)